# Western States Average Pace Prediction

For Western States, using a case base of 10 similar runners, a challenging but achievable average pace can be
produced. Thus far, the 10 most similar non PB average paces to the query runners non PB
average pace has been highlighted. The case base contains both the non PB and PB average
pace of the most similar runners. The PB average pace of these runners is then highlighted. The
runner in the case base with the fastest average pace is identified. The reason for this is that this
would output the best achievable average pace possible for each runner. To make the prediction
relative the average pace is weighted by comparing the non PB of both the query runner and
fastest case. For example, if the fastest case has a non PB of 11 minutes per kilometres, and
the query has 11.5 minutes per kilometre, the query pace is divided by the fastest case pace, and
fastest case PB is multiplied by the value. This means it caters for the fact that the similar non
PB average pace could be slightly slower or faster than the query non PB average pace. The
second type of prediction made is the mean of the PB average paces of the similar runners. Each
average pace is aggregated and divided the number of runners. This should be 10 but may be
lower for runners who do not have 10 similar runners of equal category. As previously stated,
cosine and euclidean distance are used to find the most similar runners to the query runner.
These distances were used to weight the most similar heavier than the least similar runners. For
example, the most similar runner could have a cosine similarity of 0.997 to the query runner. To
get the weight this figure is divided by the sum of all cosine similarities, and then multiplied by
the average pace. The sum of the average paces multiplied by their weights are then found. This
process is carried out for both euclidean distance values as well as cosine distance values. This
results in four predictions in total: Fastest case PB, Mean of cases PB, euclidean weighted mean
of cases PB, and cosine weighted mean of cases PB.


In [1]:
import os.path
import datetime
import pandas as pd
import numpy as np
import hashlib
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.spatial.distance import euclidean

%matplotlib inline

# Display all columns in Pandas
pd.set_option('display.max_columns', None) 

I aim to produce a challenging but not impossible average pace prediction for each runner. I will first get the runner's non personal best average pace and find the most similar non personal best average pace to that runner. I will then find the personal best average pace of these runners. By average these pb times, or getting the best personal best average pace of these runners, I will be able to produce a predicted average pace. 

In [2]:
# Constants
pace_threshold = 1
#euclidean_distance_threshold = 200

In [3]:
# Read in Split Timings
df = pd.read_csv('../../data/ws_1km_splits_full.csv', encoding = "ISO-8859-1")

In [4]:
df = df[df['Year'] != 2010]

df = df[df['Year'] != 2012]

df = df[df['Year'] != 2018]

In [5]:
# Drop unnecessary columns
df = df.drop(['Unnamed: 0', 'bib', 'rank', 'nationality'], axis = 1)

In [6]:
df.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
111,2017,10c5f67365f4b5d4974e26b9d8494f25,Male,Senior Men (23-39),35.0,6.758032,NaN,6.046816,NaN,5.144928,NaN,NaN,6.315789,NaN,4.696970,4.821429,4.352941,7.777778,4.146341,11.555556,6.234323,NaN,6.002381,NaN,NaN,5.555556,9.310345,6.321839,NaN,5.730337,NaN,7.068966,NaN,NaN,13.529412,6.484127,979.616667,6.729151
112,2017,196d4c89233fe69a5ff5b18e68440d8c,Male,Senior Men (23-39),32.0,6.848394,NaN,6.215356,NaN,5.652174,NaN,NaN,6.631579,NaN,5.000000,5.178571,4.588235,7.777778,5.487805,8.222222,5.643564,NaN,6.071429,NaN,NaN,6.324786,8.275862,7.011494,NaN,6.067416,NaN,7.931034,NaN,NaN,13.529412,5.896825,1008.383333,6.755470
113,2017,275e1385c00219e79ea54c8d6381fc07,Male,Senior Men (23-39),32.0,6.904618,NaN,6.672285,NaN,5.434783,NaN,NaN,6.842105,NaN,4.696970,5.535714,4.588235,8.055556,5.243902,9.777778,6.118812,NaN,5.942857,NaN,NaN,5.982906,7.931034,6.321839,NaN,6.179775,NaN,7.586207,NaN,NaN,13.823529,6.642857,1012.950000,6.856935
114,2017,9b61a10caab917ad2d0e7b1621007ac5,Male,Senior Men (23-39),37.0,6.860442,NaN,6.192884,NaN,5.579710,NaN,NaN,6.736842,NaN,5.000000,5.178571,4.588235,8.194444,5.609756,10.888889,6.966997,NaN,6.330952,NaN,NaN,6.068376,9.310345,7.011494,NaN,6.966292,NaN,8.103448,NaN,NaN,15.000000,6.277778,1056.183333,7.203445
115,2017,b06c1c4b8fe7d9f6f7cd49e3cba22198,Male,Senior Men (23-39),25.0,6.878514,NaN,6.720974,NaN,5.434783,NaN,NaN,7.052632,NaN,5.454545,5.714286,5.176471,8.472222,5.731707,9.111111,6.486799,NaN,5.820238,NaN,NaN,NaN,31.724138,7.126437,NaN,7.191011,NaN,8.965517,NaN,NaN,15.294118,5.325397,1057.183333,8.537828


In [7]:
df.shape

(1848, 38)

In [8]:
df.groupby('category').count()

,Year,name_mask,gender,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),588,588,588,588,549,0,574,0,562,0,0,574,0,536,521,527,541,426,525,509,0,478,0,0,466,464,462,357,85,367,86,99,0,436,447,448,584
Masters Men 2 (50-59),268,268,268,268,250,0,258,0,255,0,0,261,0,234,232,226,238,182,227,218,0,202,0,0,192,188,183,150,29,148,29,35,0,174,175,177,266
Masters Men 3 (60-69),56,56,56,56,51,0,55,0,54,0,0,53,0,50,48,50,50,35,46,43,0,41,0,0,37,37,37,32,4,30,4,8,0,33,32,32,56
Masters Men 4 (70-79),5,5,5,5,4,0,5,0,4,0,0,4,0,4,4,3,4,2,2,2,0,2,0,0,2,2,2,1,0,1,0,0,0,1,0,0,5
Masters Women 1 (40-49),145,145,145,145,126,0,136,0,137,0,0,136,0,124,122,124,129,97,126,121,0,118,0,0,111,113,112,94,13,94,13,22,0,104,106,108,143
Masters Women 2 (50-59),71,71,71,71,62,0,69,0,65,0,0,61,0,55,51,52,52,35,48,44,0,43,0,0,43,41,40,36,5,35,4,6,0,37,37,37,71
Masters Women 3 (60-69),5,5,5,5,4,0,5,0,5,0,0,5,0,3,2,3,3,3,3,3,0,3,0,0,3,2,2,1,1,1,1,0,0,2,2,2,5
Masters Women 4 (70-79),3,3,3,3,2,0,3,0,3,0,0,2,0,2,2,2,2,1,2,2,0,2,0,0,2,1,1,1,0,1,0,0,0,1,1,1,3
Senior Men (23-39),529,529,529,529,500,0,515,0,507,0,0,521,0,497,491,491,491,397,484,465,0,441,0,0,426,420,419,341,77,341,76,94,0,407,415,418,526


In [9]:
# Filter out NaN times
df = df.dropna(subset=['time'])
# Sort by name (first) and year (second)
# Sorting by year, since it might make sense to later only generate prediction for latest non-pb time
df.sort_values(['name_mask', 'Year'], ascending=[True, False], inplace=True)
# Show sample of data
df.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
1994,2015,001e405237215c01ebb54168bc0d93ec,Male,Masters Men 2 (50-59),54.0,7.951807,NaN,8.539326,NaN,7.028986,NaN,NaN,11.684211,NaN,10.909091,8.214286,8.117647,13.472222,9.756098,14.666667,11.683168,NaN,10.071429,NaN,NaN,11.880342,19.310345,10.919540,10.649351,NaN,14.259259,NaN,NaN,NaN,28.299020,8.960317,1669.033333,11.914374
1436,2014,003e885586c8ce6413604f5cc8ead272,Male,Masters Men 1 (40-49),43.0,8.132530,NaN,8.651685,NaN,7.391304,NaN,NaN,10.631579,NaN,12.121212,10.000000,8.235294,12.222222,11.219512,13.777778,11.188119,NaN,11.357143,NaN,NaN,NaN,76.551724,11.724138,11.168831,NaN,12.407407,NaN,8.378378,NaN,13.117647,8.984127,1737.466667,14.066349
124,2017,00dd2d2002b5d45f417ae37cb4bc29e3,Male,Senior Men (23-39),37.0,7.042169,NaN,6.640449,NaN,6.014493,NaN,NaN,6.947368,NaN,5.909091,5.714286,5.176471,10.000000,6.463415,11.555556,7.632013,NaN,7.422619,NaN,NaN,9.914530,9.310345,8.160920,NaN,10.786517,NaN,10.862069,NaN,NaN,20.294118,10.904762,1262.900000,8.776378
1134,2016,0104b6fc44ae66e38da37734f708ab1f,Male,Masters Men 1 (40-49),42.0,7.464859,NaN,7.554307,NaN,6.221014,NaN,NaN,9.473684,NaN,9.696970,6.785714,6.352941,10.972222,NaN,29.333333,8.480198,NaN,8.453571,NaN,NaN,7.863248,13.448276,8.620690,8.831169,NaN,10.555556,NaN,NaN,NaN,NaN,48.007937,1369.816667,12.242099
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.584270,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747


# Filter out runners who only ran one race

Runner's have ran at least 2 races to be included in the case base, ie. a non personal best time and a personal best time

In [10]:
# Count number of races per runner
v = df.name_mask.value_counts()
# Get runners who ran more than one race
multi_race_runners = v.index[v.gt(1)]
# Filter by multi_race_runners
df = df[df.name_mask.isin(multi_race_runners)]

In [11]:
df.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
1018,2017,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.58427,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579
1378,2014,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),47.0,8.313253,NaN,8.314607,NaN,7.318841,NaN,NaN,9.894737,NaN,9.242424,7.142857,5.882353,12.222222,6.951220,13.777778,8.811881,NaN,8.214286,NaN,NaN,7.948718,12.758621,7.701149,7.272727,NaN,11.111111,NaN,7.567568,NaN,12.323529,8.269841,1397.266667,9.051986
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974


# Merge

In [12]:
# Get Personal Best
pb = df[df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]
# Drop Year and category to avoid duplication on merge
pb = pb.drop(['Year', 'category'], axis = 1)
# Get Non-Personal Best Times
npb = df[~df.time.groupby(df.name_mask).apply(lambda x: x == x.min())]

In [13]:
pb.head()

,name_mask,gender,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
1018,0106a626dcce58322c9f90dfcd777b9c,Male,42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.58427,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747
1378,017b3bcb28ddc8e5336d606e57611f84,Male,47.0,8.313253,NaN,8.314607,NaN,7.318841,NaN,NaN,9.894737,NaN,9.242424,7.142857,5.882353,12.222222,6.951220,13.777778,8.811881,NaN,8.214286,NaN,NaN,7.948718,12.758621,7.701149,7.272727,NaN,11.111111,NaN,7.567568,NaN,12.323529,8.269841,1397.266667,9.051986
2043,0530f3b9ddc07dcdff4e1363db2b13ad,Male,51.0,8.554217,NaN,8.651685,NaN,7.246377,NaN,NaN,10.315789,NaN,10.303030,9.464286,NaN,19.444444,8.292683,12.888889,9.108911,NaN,8.642857,NaN,NaN,NaN,48.965517,10.344828,11.948052,NaN,13.703704,NaN,NaN,NaN,26.156863,6.484127,1517.550000,13.559780
2052,066dd61e63158704bcb0e6ee2bd5991e,Male,51.0,7.951807,NaN,8.202247,NaN,7.391304,NaN,NaN,10.421053,NaN,10.909091,7.678571,NaN,22.361111,10.731707,14.000000,10.594059,NaN,10.785714,NaN,NaN,11.196581,15.517241,11.264368,12.727273,NaN,13.888889,NaN,9.729730,NaN,10.627451,8.785714,1664.583333,11.303364
2481,0a43ae8070c7d12e36c34816c6278951,Female,34.0,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,1267.666667,9.710111


In [14]:
npb.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
1275,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720
1158,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579
1985,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974
1996,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),52.0,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,1681.783333,11.844286
2510,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),33.0,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,1307.416667,9.080386


In [15]:
# Merge to get non-pb runs with personal best time (with time, timediff and AVG_Pace)
merge = npb.merge(pb,how='left',on='name_mask',suffixes=('', '_pb'))

In [16]:
merge.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720,Male,42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.58427,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579,Male,47.0,8.313253,NaN,8.314607,NaN,7.318841,NaN,NaN,9.894737,NaN,9.242424,7.142857,5.882353,12.222222,6.951220,13.777778,8.811881,NaN,8.214286,NaN,NaN,7.948718,12.758621,7.701149,7.272727,NaN,11.111111,NaN,7.567568,NaN,12.323529,8.269841,1397.266667,9.051986
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974,Male,51.0,8.554217,NaN,8.651685,NaN,7.246377,NaN,NaN,10.315789,NaN,10.303030,9.464286,NaN,19.444444,8.292683,12.888889,9.108911,NaN,8.642857,NaN,NaN,NaN,48.965517,10.344828,11.948052,NaN,13.703704,NaN,NaN,NaN,26.156863,6.484127,1517.550000,13.559780
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),52.0,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.

In [17]:
merge.groupby('category').count()

,Year,name_mask,gender,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Masters Men 1 (40-49),51,51,51,51,48,0,49,0,51,0,0,51,0,49,45,50,50,39,51,51,0,48,0,0,50,50,51,37,13,38,13,5,0,49,51,51,51,51,51,48,0,51,0,43,0,0,51,0,51,47,50,51,45,51,51,0,50,0,0,47,50,51,46,5,46,5,24,0,51,51,51,51
Masters Men 2 (50-59),22,22,22,22,22,0,21,0,21,0,0,22,0,22,21,21,22,16,22,22,0,22,0,0,21,22,22,15,7,15,7,3,0,21,22,22,22,22,22,21,0,22,0,22,0,0,22,0,22,22,19,22,20,22,22,0,22,0,0,21,21,22,19,1,21,1,7,0,22,22,22,22
Masters Men 3 (60-69),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,2,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,5,0,5,0,4,0,0,5,0,5,5,5,5,5,5,5,0,5,0,0,4,5,5,5,0,5,0,1,0,5,5,5,5
Masters Women 1 (40-49),12,12,12,12,11,0,12,0,11,0,0,12,0,12,11,12,12,10,12,12,0,12,0,0,12,12,12,11,1,11,1,2,0,12,12,12,12,12,12,12,0,11,0,12,0,0,12,0,12,12,12,12,12,12,12,0,12,0,0,12,12,12,10,2,9,2,3,0,12,12,12,12
Masters Women 2 (50-59),5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,4,1,4,1,1,0,5,5,5,5,5,5,5,0,5,0,5,0,0,5,0,5,5,5,5,4,5,5,0,5,0,0,5,5,5,5,0,5,0,0,0,5,5,5,5
Senior Men (23-39),44,44,44,44,43,0,43,0,44,0,0,44,0,42,43,41,43,39,44,44,0,44,0,0,43,43,44,32,12,32,12,9,0,44,44,44,44,44,44,44,0,44,0,43,0,0,44,0,44,44,41,44,34,44,44,0,42,0,0,44,44,44,38,6,38,6,15,0,42,44,44,44
Senior Women (23-39),14,14,14,14,13,0,14,0,12,0,0,14,0,14,14,14,14,13,14,14,0,13,0,0,14,14,14,11,3,11,3,4,0,14,14,14,14,14,14,13,0,14,0,14,0,0,14,0,14,14,14,14,6,14,14,0,14,0,0,14,14,14,14,0,13,0,1,0,14,14,14,14


#merge = merge[merge.category != 'Youth Men (20-22)']
merge = merge[merge.category != 'Masters Women 3 (60-69)']
#merge = merge[merge.category != 'Masters Men 4 (70-79)']

merge = merge[merge.name_mask != '3bb1e8b42dd9314fd4d8549f31d001f3']
merge = merge[merge.name_mask != '4d5f9076cc4be3cdf23a51be2c1e7d8c']
merge = merge[merge.name_mask != '738138c60e7b32635a37f6d0757c1512']
merge = merge[merge.name_mask != '972253ab3ee99e5c6bcee7a8eba23ebf']
merge = merge[merge.name_mask != 'fbac5ac0b66892696a502c9f3de3ee0e']

In [18]:
merge = merge[merge.name_mask != 'f74186d56d674bb6eb38a44bd8a579a3']

In [19]:
merge = merge.reset_index(drop=True)

In [20]:
cats = merge['category']
gen = merge['gender']
comb = [cats, gen]
comb = pd.concat(comb, axis=1)

In [21]:
# Update pb and npb to use merged indices
# pb does not contain name_mask, category or year
pb = merge.loc[:, merge.columns.str.contains('pb')]
npb = merge.loc[:, ~merge.columns.str.contains('pb')]

In [22]:
merge.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720,Male,42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.58427,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579,Male,47.0,8.313253,NaN,8.314607,NaN,7.318841,NaN,NaN,9.894737,NaN,9.242424,7.142857,5.882353,12.222222,6.951220,13.777778,8.811881,NaN,8.214286,NaN,NaN,7.948718,12.758621,7.701149,7.272727,NaN,11.111111,NaN,7.567568,NaN,12.323529,8.269841,1397.266667,9.051986
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974,Male,51.0,8.554217,NaN,8.651685,NaN,7.246377,NaN,NaN,10.315789,NaN,10.303030,9.464286,NaN,19.444444,8.292683,12.888889,9.108911,NaN,8.642857,NaN,NaN,NaN,48.965517,10.344828,11.948052,NaN,13.703704,NaN,NaN,NaN,26.156863,6.484127,1517.550000,13.559780
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),52.0,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.

In [23]:
pb.head()

,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,Male,42.0,8.718876,NaN,8.344569,NaN,6.594203,NaN,NaN,9.578947,NaN,7.727273,7.142857,6.705882,12.083333,8.536585,13.777778,8.879538,NaN,8.808333,NaN,NaN,9.401709,14.827586,10.000000,NaN,12.58427,NaN,13.965517,NaN,NaN,24.117647,9.714286,1516.400000,10.605747
1,Male,47.0,8.313253,NaN,8.314607,NaN,7.318841,NaN,NaN,9.894737,NaN,9.242424,7.142857,5.882353,12.222222,6.951220,13.777778,8.811881,NaN,8.214286,NaN,NaN,7.948718,12.758621,7.701149,7.272727,NaN,11.111111,NaN,7.567568,NaN,12.323529,8.269841,1397.266667,9.051986
2,Male,51.0,8.554217,NaN,8.651685,NaN,7.246377,NaN,NaN,10.315789,NaN,10.303030,9.464286,NaN,19.444444,8.292683,12.888889,9.108911,NaN,8.642857,NaN,NaN,NaN,48.965517,10.344828,11.948052,NaN,13.703704,NaN,NaN,NaN,26.156863,6.484127,1517.550000,13.559780
3,Male,51.0,7.951807,NaN,8.202247,NaN,7.391304,NaN,NaN,10.421053,NaN,10.909091,7.678571,NaN,22.361111,10.731707,14.000000,10.594059,NaN,10.785714,NaN,NaN,11.196581,15.517241,11.264368,12.727273,NaN,13.888889,NaN,9.729730,NaN,10.627451,8.785714,1664.583333,11.303364
4,Female,34.0,7.902610,NaN,7.764045,NaN,6.283816,NaN,NaN,8.842105,NaN,7.878788,6.428571,5.764706,10.000000,NaN,24.000000,7.613861,NaN,7.435714,NaN,NaN,7.692308,11.724138,8.160920,7.662338,NaN,9.814815,NaN,NaN,NaN,22.352941,7.460317,1267.666667,9.710111


In [24]:
pb.to_csv('../../data/ws1_pb_times.csv')

In [25]:
npb.head()

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),52.0,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,1681.783333,11.844286
4,2015,0a43ae8070c7d12e36c34816c6278951,Female,Senior Women (23-39),33.0,8.149598,NaN,8.282772,NaN,6.449275,NaN,NaN,10.315789,NaN,7.424242,6.785714,6.000000,10.277778,7.195122,12.444444,7.722772,NaN,7.285714,NaN,NaN,8.205128,11.724138,8.620690,8.051948,NaN,9.444444,NaN,NaN,NaN,21.004902,7.142857,1307.416667,9.080386


# Define Functions

In [26]:
def find_similar_times(index):
    query = npb.iloc[index]
    same_gender = npb[npb.gender == query.gender]
    similar_times = same_gender[(same_gender.AVG_Pace > query.AVG_Pace - pace_threshold) & (same_gender.AVG_Pace < query.AVG_Pace + pace_threshold)]
    #find all runners either 0.5 minutes/kilometre faster or slower than the query runner
   # if len(similar_times) = 1:
    #    similar_times
    return similar_times
    #else:
     #   return None

# Show sample output
find_similar_times(0)

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace
0,2015,0106a626dcce58322c9f90dfcd777b9c,Male,Masters Men 1 (40-49),40.0,9.037149,NaN,9.548689,NaN,7.246377,NaN,NaN,10.000000,NaN,10.606061,8.928571,7.529412,14.166667,9.756098,14.000000,9.603960,NaN,10.500000,NaN,NaN,12.051282,15.862069,10.574713,11.688312,NaN,14.074074,NaN,NaN,NaN,29.735294,9.380952,1668.800000,11.804720
1,2016,017b3bcb28ddc8e5336d606e57611f84,Male,Masters Men 1 (40-49),49.0,9.353414,NaN,8.788390,NaN,7.283816,NaN,NaN,11.052632,NaN,10.303030,8.392857,6.941176,13.194444,NaN,32.444444,11.343234,NaN,11.316667,NaN,NaN,10.256410,14.137931,9.885057,10.519481,NaN,12.407407,NaN,NaN,NaN,25.000000,7.746032,1623.266667,12.242579
2,2015,0530f3b9ddc07dcdff4e1363db2b13ad,Male,Masters Men 2 (50-59),52.0,8.421687,NaN,8.449438,NaN,7.463768,NaN,NaN,11.473684,NaN,10.606061,7.857143,7.529412,13.472222,10.853659,14.666667,11.782178,NaN,9.214286,NaN,NaN,9.487179,13.793103,9.885057,9.610390,NaN,11.296296,NaN,NaN,NaN,23.225490,8.277778,1573.350000,10.913974
3,2015,066dd61e63158704bcb0e6ee2bd5991e,Male,Masters Men 2 (50-59),52.0,8.568273,NaN,9.187266,NaN,7.246377,NaN,NaN,10.421053,NaN,10.303030,7.857143,7.764706,13.611111,8.902439,17.111111,14.158416,NaN,10.214286,NaN,NaN,13.247863,18.620690,10.114943,9.870130,NaN,11.481481,NaN,NaN,NaN,27.250000,9.111111,1681.783333,11.844286
16,2015,1cfacfe60837958f1be7a2b13aaac0b5,Male,Senior Men (23-39),34.0,9.295181,NaN,8.842697,NaN,6.594203,NaN,NaN,10.947368,NaN,10.000000,8.035714,8.235294,13.472222,9.390244,14.222222,10.594059,NaN,8.428571,NaN,NaN,10.683761,16.896552,9.080460,9.480519,NaN,12.962963,NaN,NaN,NaN,27.196078,9.722222,1580.883333,11.267386
17,2016,1ea16509e1c34400de4a0af729413bb8,Male,Masters Men 1 (40-49),48.0,8.086345,NaN,7.975655,NaN,6.868357,NaN,NaN,10.947368,NaN,10.606061,8.214286,8.117647,12.361111,NaN,30.666667,10.463696,NaN,9.665476,NaN,NaN,13.931624,14.482759,12.528736,11.298701,NaN,13.333333,NaN,NaN,NaN,31.176471,8.269841,1653.366667,12.721896
18,2013,1fa40875748bfac9c9439865a1c7efc7,Male,Masters Men 1 (40-49),40.0,6.325301,NaN,7.078652,NaN,6.014493,NaN,NaN,9.578947,NaN,10.606061,8.392857,6.235294,14.861111,10.609756,14.000000,8.118812,NaN,7.928571,NaN,NaN,10.256410,15.862069,9.655172,13.506494,NaN,13.148148,NaN,NaN,NaN,29.705882,10.111111,1509.233333,11.157639
27,2013,3f2963c8306fa48b5550122a05d185b5,Male,Masters Men 1 (40-49),45.0,10.060241,NaN,12.022472,NaN,8.695652,NaN,NaN,11.368421,NaN,11.666667,9.464286,9.176471,6.111111,17.804878,14.222222,10.891089,NaN,11.000000,NaN,NaN,12.136752,15.862069,13.218391,10.389610,NaN,11.666667,NaN,NaN,NaN,28.235294,10.039683,1791.083333,12.317472
28,2015,420266030cd34eb1924b8a01967ce661,Male,Senior Men (23-39),34.0,8.222892,NaN,7.696629,NaN,6.304348,NaN,NaN,8.736842,NaN,8.484848,6.785714,6.470588,12.777778,8.536585,15.333333,9.207921,NaN,9.214286,NaN,NaN,12.136752,14.827586,14.022989,14.545455,NaN,16.666667,NaN,NaN,NaN,37.019608,10.388889,1633.683333,11.967353
30,2017,47b73fd646234456b1d7cc2352e16e0e,Male,Masters Men 1 (40-49),41.0,8.804217,NaN,9.084270,NaN,7.173913,NaN,NaN,10.421053,NaN,8.7878

In [27]:
def cosine_similarity(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    # Return cosine similarity (1 - cosine difference)
    return 1 - (cosine(a, b))

def sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: cosine_similarity(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = False) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

sims(0)

70     0.997827
16     0.996600
96     0.996336
134    0.995625
149    0.995271
18     0.994524
146    0.994441
122    0.994285
135    0.993838
108    0.992714
dtype: float64

In [28]:
def euc_distance(a, b):
    # Combine two lists (row and query run) into matrix
    matrix = pd.DataFrame({"A": a, "B": b})
    # Now drop any pairs containing NA so we only compare when we have two times
    matrix = matrix.dropna(axis = 0, how='any')
    # Parse back to two lists
    a = matrix[['A']]
    b = matrix[['B']]
    
    return euclidean(a,b)

def euc_sims(index):
    similar_times = find_similar_times(index)
    #similar_times = similar_times.dropna(how='all')
    #if sim_count > 1:
    similar_times = similar_times.filter(like='to')
    #if similar_times.count() > 1:
    query_segs = list(similar_times.loc[index])
    similar_times = similar_times.drop([index], axis = 0)
    result = similar_times.apply(lambda row: euc_distance(list(row), query_segs), axis = 1)
    result = result.sort_values(ascending = True) #finding the most similar
    result = result.head(10) #10 is the number of neighbours as an example, the optimal value for n is tested in Prediction_Eval notebook
    
    return result

    #elif sim_count <= 1:
     #   return 0

euc_sims(30)

15     10.683798
39     13.011692
129    13.337466
98     13.889217
108    13.938887
93     14.505685
69     14.757024
120    14.908144
100    15.078731
37     15.090692
dtype: float64

In [29]:
def weight(index):
    weighter = sims(index)
    #weight_count = len(weighter)
    #weight0 = len(str(weight_count))
   # if weight_count > 1:
   # if weighter.sum() > 3:
    if len(weighter) > 1:
        weighter = weighter / weighter.sum()
        return weighter
    
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
    #else:
     #   return np.nan
    
    #elif weight0 <= 1:
     #   return 0
    

weight(30)

15     0.101521
129    0.100450
39     0.100238
37     0.099929
93     0.099838
120    0.099726
98     0.099693
69     0.099629
108    0.099619
146    0.099356
dtype: float64

In [30]:
def euc_weight(index):
    weighter = euc_sims(index)
    if len(weighter) > 1:
        weighter = 1 -(weighter / weighter.sum())
        weighter = weighter / weighter.sum()
        return weighter
    else:
        weighter1 = weighter / weighter
        return weighter1 / 10
    
euc_weight(66)

141    0.116048
144    0.115884
41     0.115035
31     0.112271
5      0.109737
151    0.109341
42     0.109128
21     0.108579
63     0.103977
dtype: float64

In [31]:
def find_nearest_neighbours(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    if len(nearest_neighbours) > 1:
        nearest_neighbours['weights'] = weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    else: 
        nearest_neighbours['weights'] = weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
        
        return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours(0)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb,weights,weighted_pace
70,2015,758ede5c3802b99099cee83399a2c5aa,Male,Masters Men 2 (50-59),50.0,8.146586,NaN,8.850187,NaN,6.666667,NaN,NaN,10.736842,NaN,9.696970,8.571429,7.647059,13.888889,8.292683,14.666667,10.594059,NaN,10.142857,NaN,NaN,11.709402,16.206897,11.839080,11.948052,NaN,14.629630,NaN,NaN,NaN,30.294118,11.563492,1653.283333,11.899556,Male,48.0,8.072289,NaN,8.089888,NaN,6.521739,NaN,NaN,9.578947,NaN,8.939394,7.500000,6.117647,11.388889,7.439024,13.111111,9.009901,NaN,7.642857,NaN,NaN,9.230769,15.517241,12.528736,NaN,NaN,33.518519,NaN,NaN,NaN,36.470588,13.579365,1535.516667,12.458717,0.100269,1.249228
16,2015,1cfacfe60837958f1be7a2b13aaac0b5,Male,Senior Men (23-39),34.0,9.295181,NaN,8.842697,NaN,6.594203,NaN,NaN,10.947368,NaN,10.000000,8.035714,8.235294,13.472222,9.390244,14.222222,10.594059,NaN,8.428571,NaN,NaN,10.683761,16.896552,9.080460,9.480519,NaN,12.962963,NaN,NaN,NaN,27.196078,9.722222,1580.883333,11.267386,Male,36.0,9.129518,NaN,7.578652,NaN,6.956522,NaN,NaN,10.947368,NaN,8.333333,6.785714,7.058824,10.555556,7.560976,14.666667,12.214521,NaN,9.616667,NaN,NaN,10.000000,12.758621,8.850575,NaN,9.438202,NaN,10.517241,NaN,NaN,18.823529,10.134921,1495.283333,10.101442,0.100146,1.011620
96,2015,b073a967413c1f902bdbd5efb9885527,Male,Masters Men 1 (40-49),45.0,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,1597.733333,11.130149,Male,47.0,8.912651,NaN,8.882022,NaN,7.101449,NaN,NaN,9.894737,NaN,7.575758,7.321429,6.823529,11.111111,7.439024,12.444444,7.849835,NaN,8.122619,NaN,NaN,7.777778,11.034483,8.735632,NaN,8.539326,NaN,9.655172,NaN,NaN,16.470588,7.317460,1360.366667,9.105739,0.100120,0.911663
134,2013,dd70016790840372dd12d079802fe393,Male,Masters Men 1 (40-49),41.0,9.518072,NaN,9.887640,NaN,7.826087,NaN,NaN,10.631579,NaN,10.606061,10.000000,7.764706,13.055556,10.121951,10.

In [32]:
def euc_find_nearest_neighbours(index):
    #w = weight(index)
    euc_distance = euc_weight(index)  
    nearest_neighbours = merge.iloc[euc_distance.index]
    if len(nearest_neighbours) > 1: 
        nearest_neighbours['weights'] = euc_weight(index)
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
    else: 
        nearest_neighbours['weights'] = euc_weight(index) * 10
        nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb'] 
   # nearest_neighbours['rank_weights'] = rank_weight(index)
   # nearest_neighbours['ranked_pace'] = nearest_neighbours['rank_weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
        return nearest_neighbours
    
# Show sample output    
euc_find_nearest_neighbours(66)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb,weights,weighted_pace
141,2015,e8fce977f0ecd7532278e7c0ff2c35ff,Female,Masters Women 2 (50-59),54.0,7.590361,NaN,7.752809,NaN,6.159420,NaN,NaN,9.052632,NaN,8.030303,6.607143,6.000000,10.694444,6.951220,12.888889,8.019802,NaN,7.428571,NaN,NaN,8.205128,11.724138,9.770115,9.480519,NaN,11.296296,NaN,NaN,NaN,30.715686,9.214286,1356.783333,9.872724,Female,52.0,6.987952,NaN,7.303371,NaN,5.507246,NaN,NaN,7.684211,NaN,7.575758,6.250000,5.294118,9.305556,6.097561,11.777778,7.425743,NaN,7.071429,NaN,NaN,7.008547,10.344828,7.586207,7.142857,NaN,9.444444,NaN,NaN,NaN,19.705882,7.539683,1170.833333,8.265956,0.116048,0.959250
144,2015,f84f940684e04bbb773ae4a48626f5e6,Female,Masters Women 1 (40-49),40.0,7.469880,NaN,7.752809,NaN,6.086957,NaN,NaN,8.315789,NaN,8.030303,5.892857,5.529412,9.722222,6.585366,13.111111,8.415842,NaN,7.000000,NaN,NaN,8.290598,13.103448,10.574713,11.818182,NaN,14.074074,NaN,NaN,NaN,29.372549,8.309524,1366.316667,9.971349,Female,38.0,6.626506,NaN,6.966292,NaN,5.579710,NaN,NaN,7.578947,NaN,7.272727,6.250000,4.941176,8.611111,6.219512,10.000000,6.633663,NaN,6.285714,NaN,NaN,6.666667,10.000000,7.241379,7.142857,NaN,NaN,NaN,NaN,NaN,35.000000,6.833333,1117.350000,8.658311,0.115884,1.003363
41,2015,55f4a50d0afd8466fd4eb7443eb6df4f,Female,Masters Women 1 (40-49),45.0,7.349398,NaN,7.303371,NaN,5.869565,NaN,NaN,8.631579,NaN,8.636364,6.785714,6.117647,12.361111,9.512195,17.111111,12.079208,NaN,8.142857,NaN,NaN,9.658120,11.034483,7.931034,8.441558,NaN,11.851852,NaN,NaN,NaN,25.362745,7.357143,1421.683333,10.080898,Female,44.0,7.168675,NaN,6.966292,NaN,5.652174,NaN,NaN,7.684211,NaN,7.575758,5.535714,5.647059,9.583333,6.463415,10.888889,7.623762,NaN,7.142857,NaN,NaN,7.863248,10.689655,7.701149,7.662338,NaN,11.111111,NaN,6.351351,NaN,10.612745,8.769841,1219.500000,7.934679,0.115035,0.912766
31,2013,498edc8f8acd17ca7e59076d4cebe061,Female,Masters Women 1 (40-49),42.0,8.674699,NaN,8.426966,NaN,NaN,NaN,NaN,18.210526,NaN,8.939394,7.678571,5.882353,10.555556,8.902439,12.22222

In [33]:
def find_nearest_neighbours_pace(index):
    #w = weight(index)
    cosine_distance = weight(index)  
    nearest_neighbours = merge.iloc[cosine_distance.index]
    #nearest_neighbours = nearest_neighbours.filter(like='to')
    #nearest_neighbours['weights'] = weight(index)
    #nearest_neighbours['weighted_pace'] = nearest_neighbours['weights'] * nearest_neighbours['AVG_Pace_pb']
    #nearest_neighbours = nearest_neighbours.loc[:, 'Chamonix to Delevret_pb':'Flegere to Chamonix_pb'].mul(nearest_neighbours['weighted_pace'], axis=0)
    #nearest_neighbours = npb.iloc[euclidean_distance.index]
    return nearest_neighbours
    
# Show sample output    
find_nearest_neighbours_pace(5)

,Year,name_mask,gender,category,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,gender_pb,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
141,2015,e8fce977f0ecd7532278e7c0ff2c35ff,Female,Masters Women 2 (50-59),54.0,7.590361,NaN,7.752809,NaN,6.159420,NaN,NaN,9.052632,NaN,8.030303,6.607143,6.000000,10.694444,6.951220,12.888889,8.019802,NaN,7.428571,NaN,NaN,8.205128,11.724138,9.770115,9.480519,NaN,11.296296,NaN,NaN,NaN,30.715686,9.214286,1356.783333,9.872724,Female,52.0,6.987952,NaN,7.303371,NaN,5.507246,NaN,NaN,7.684211,NaN,7.575758,6.250000,5.294118,9.305556,6.097561,11.777778,7.425743,NaN,7.071429,NaN,NaN,7.008547,10.344828,7.586207,7.142857,NaN,9.444444,NaN,NaN,NaN,19.705882,7.539683,1170.833333,8.265956
41,2015,55f4a50d0afd8466fd4eb7443eb6df4f,Female,Masters Women 1 (40-49),45.0,7.349398,NaN,7.303371,NaN,5.869565,NaN,NaN,8.631579,NaN,8.636364,6.785714,6.117647,12.361111,9.512195,17.111111,12.079208,NaN,8.142857,NaN,NaN,9.658120,11.034483,7.931034,8.441558,NaN,11.851852,NaN,NaN,NaN,25.362745,7.357143,1421.683333,10.080898,Female,44.0,7.168675,NaN,6.966292,NaN,5.652174,NaN,NaN,7.684211,NaN,7.575758,5.535714,5.647059,9.583333,6.463415,10.888889,7.623762,NaN,7.142857,NaN,NaN,7.863248,10.689655,7.701149,7.662338,NaN,11.111111,NaN,6.351351,NaN,10.612745,8.769841,1219.500000,7.934679
144,2015,f84f940684e04bbb773ae4a48626f5e6,Female,Masters Women 1 (40-49),40.0,7.469880,NaN,7.752809,NaN,6.086957,NaN,NaN,8.315789,NaN,8.030303,5.892857,5.529412,9.722222,6.585366,13.111111,8.415842,NaN,7.000000,NaN,NaN,8.290598,13.103448,10.574713,11.818182,NaN,14.074074,NaN,NaN,NaN,29.372549,8.309524,1366.316667,9.971349,Female,38.0,6.626506,NaN,6.966292,NaN,5.579710,NaN,NaN,7.578947,NaN,7.272727,6.250000,4.941176,8.611111,6.219512,10.000000,6.633663,NaN,6.285714,NaN,NaN,6.666667,10.000000,7.241379,7.142857,NaN,NaN,NaN,NaN,NaN,35.000000,6.833333,1117.350000,8.658311
66,2017,6efa98760653a5b256fefc810072ecea,Female,Senior Women (23-39),31.0,8.317269,NaN,7.520599,NaN,6.159420,NaN,NaN,7.789474,NaN,5.757576,5.535714,5.176471,11.111111,10.853659,13.111111,14.508251,NaN,10.961905,NaN,NaN,9.401709,13.793103,10.574713,NaN,9.21348

In [34]:
def pace_rec(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

pace_rec(0)

,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,2014.8,43.6,8.752711,NaN,9.023034,NaN,7.275362,NaN,NaN,10.789474,NaN,10.121212,8.553571,7.635294,13.694444,9.780488,14.066667,9.779373,NaN,9.973452,NaN,NaN,11.102564,15.586207,10.747126,10.730519,12.022472,12.967207,13.87931,NaN,NaN,27.412255,9.151587,1629.238333,11.451756,44.1,8.771687,NaN,8.748689,NaN,7.272678,NaN,NaN,11.652632,NaN,9.121212,7.477679,8.623529,12.166667,8.414634,15.266667,10.049505,NaN,9.347751,NaN,NaN,10.574217,15.275862,10.091954,10.649351,9.073034,15.617284,10.387931,7.432432,NaN,18.077941,9.039683,1535.448333,10.744975


In [35]:
def weighted_rows(index):
    #print(index)
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.mean()
    row_pace = pd.DataFrame(row_pace)
    row_pace = row_pace.T
    row_pace.index = query.index
    #pred_pace = row_pace
    return row_pace

#neighbours = find_nearest_neighbours_pace(60)
#weighted_row(neighbours)
#tester = find_nearest_neighbours_pace(60)
#weighted_rows(tester)

weighted_rows(0)

,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,2014.8,43.6,8.752711,NaN,9.023034,NaN,7.275362,NaN,NaN,10.789474,NaN,10.121212,8.553571,7.635294,13.694444,9.780488,14.066667,9.779373,NaN,9.973452,NaN,NaN,11.102564,15.586207,10.747126,10.730519,12.022472,12.967207,13.87931,NaN,NaN,27.412255,9.151587,1629.238333,11.451756,44.1,8.771687,NaN,8.748689,NaN,7.272678,NaN,NaN,11.652632,NaN,9.121212,7.477679,8.623529,12.166667,8.414634,15.266667,10.049505,NaN,9.347751,NaN,NaN,10.574217,15.275862,10.091954,10.649351,9.073034,15.617284,10.387931,7.432432,NaN,18.077941,9.039683,1535.448333,10.744975


In [36]:
weighted_rows(4)

,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
4,2014.8,41.9,7.353815,NaN,7.373783,NaN,5.990338,NaN,NaN,9.336842,NaN,7.621212,6.071429,5.647059,9.722222,6.731707,12.088889,7.871452,NaN,7.257024,NaN,NaN,7.649573,11.62069,8.908046,8.863636,7.865169,10.787037,9.741379,7.837838,NaN,20.589216,8.01746,1269.44,8.905557,41.4,6.821386,NaN,6.946816,NaN,5.517271,NaN,NaN,7.568421,NaN,7.545455,5.982143,5.258824,9.083333,6.324042,14.183333,7.283003,NaN,6.929762,NaN,NaN,6.923077,9.896552,7.655172,7.311688,NaN,9.588477,NaN,NaN,NaN,21.956373,7.126984,1156.018333,8.450547


####  Row Prediction

In [37]:
pb1 = pb.filter(like='to')

In [38]:
pb1.shape

(153, 31)

In [39]:
weighted_pb = pd.DataFrame()

In [40]:
weighted_pb

""


In [41]:
pb1.shape[0]

153

In [42]:
for i in range(pb1.shape[0]):
    weighted_pb = weighted_pb.append(pace_rec(i), ignore_index=True)

In [43]:
weighted_pb

,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,2014.800000,43.600000,8.752711,NaN,9.023034,NaN,7.275362,NaN,NaN,10.789474,NaN,10.121212,8.553571,7.635294,13.694444,9.780488,14.066667,9.779373,NaN,9.973452,NaN,NaN,11.102564,15.586207,10.747126,10.730519,12.022472,12.967207,13.879310,NaN,NaN,27.412255,9.151587,1629.238333,11.451756,44.100000,8.771687,NaN,8.748689,NaN,7.272678,NaN,NaN,11.652632,NaN,9.121212,7.477679,8.623529,12.166667,8.414634,15.266667,10.049505,NaN,9.347751,NaN,NaN,10.574217,15.275862,10.091954,10.649351,9.073034,15.617284,10.387931,7.432432,NaN,18.077941,9.039683,1535.448333,10.744975
1,2015.900000,45.900000,8.671185,NaN,8.621723,NaN,7.153986,NaN,NaN,10.557895,NaN,10.303030,8.214286,9.717647,12.694444,10.121951,26.022222,10.358911,NaN,9.719643,NaN,NaN,11.931624,15.241379,11.034483,10.245310,11.235955,12.037037,14.137931,NaN,NaN,26.307353,8.470635,1606.733333,12.152192,46.600000,8.452588,NaN,10.268727,NaN,7.197585,NaN,NaN,10.473684,NaN,9.939394,7.757937,8.248366,13.333333,8.685637,15.555556,9.798515,NaN,9.173929,NaN,NaN,9.781576,18.344828,10.045977,10.259740,10.842697,12.129630,12.500000,7.891892,NaN,16.077941,8.239683,1535.968333,10.748605
2,2015.400000,46.700000,9.154618,NaN,9.397004,NaN,7.644928,NaN,NaN,11.200000,NaN,10.424242,8.392857,7.988235,13.541667,9.609756,14.511111,10.524587,NaN,10.257262,NaN,NaN,11.153846,15.620690,10.448276,10.204082,10.861423,12.010582,12.356322,NaN,NaN,24.804412,8.827778,1643.173333,11.369308,46.100000,8.677309,NaN,8.711236,NaN,7.321601,NaN,NaN,13.642105,NaN,10.106061,8.035714,7.947712,13.375000,8.970190,15.600000,10.039439,NaN,9.903439,NaN,NaN,11.206695,16.275862,10.574713,11.060606,9.325843,15.529101,10.344828,8.333333,NaN,20.325490,9.581746,1579.528333,11.401750
3,2015.400000,44.100000,8.772490,NaN,8.986142,NaN,7.340580,NaN,NaN,11.168421,NaN,10.090909,8.178571,7.741176,13.527778,9.524390,14.688889,10.220792,NaN,10.055000,NaN,NaN,10.905983,15.517241,10.586207,10.568182,11.348315,12.569444,14.310345,NaN,NaN,25.816667,8.879365,1624.123333,11.349602,43.800000,8.675703,NaN,8.613109,NaN,7.079442,N

In [44]:
weighted_pb = comb.join(weighted_pb)

In [45]:
weighted_pb

,category,gender,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
0,Masters Men 1 (40-49),Male,2014.800000,43.600000,8.752711,NaN,9.023034,NaN,7.275362,NaN,NaN,10.789474,NaN,10.121212,8.553571,7.635294,13.694444,9.780488,14.066667,9.779373,NaN,9.973452,NaN,NaN,11.102564,15.586207,10.747126,10.730519,12.022472,12.967207,13.879310,NaN,NaN,27.412255,9.151587,1629.238333,11.451756,44.100000,8.771687,NaN,8.748689,NaN,7.272678,NaN,NaN,11.652632,NaN,9.121212,7.477679,8.623529,12.166667,8.414634,15.266667,10.049505,NaN,9.347751,NaN,NaN,10.574217,15.275862,10.091954,10.649351,9.073034,15.617284,10.387931,7.432432,NaN,18.077941,9.039683,1535.448333,10.744975
1,Masters Men 1 (40-49),Male,2015.900000,45.900000,8.671185,NaN,8.621723,NaN,7.153986,NaN,NaN,10.557895,NaN,10.303030,8.214286,9.717647,12.694444,10.121951,26.022222,10.358911,NaN,9.719643,NaN,NaN,11.931624,15.241379,11.034483,10.245310,11.235955,12.037037,14.137931,NaN,NaN,26.307353,8.470635,1606.733333,12.152192,46.600000,8.452588,NaN,10.268727,NaN,7.197585,NaN,NaN,10.473684,NaN,9.939394,7.757937,8.248366,13.333333,8.685637,15.555556,9.798515,NaN,9.173929,NaN,NaN,9.781576,18.344828,10.045977,10.259740,10.842697,12.129630,12.500000,7.891892,NaN,16.077941,8.239683,1535.968333,10.748605
2,Masters Men 2 (50-59),Male,2015.400000,46.700000,9.154618,NaN,9.397004,NaN,7.644928,NaN,NaN,11.200000,NaN,10.424242,8.392857,7.988235,13.541667,9.609756,14.511111,10.524587,NaN,10.257262,NaN,NaN,11.153846,15.620690,10.448276,10.204082,10.861423,12.010582,12.356322,NaN,NaN,24.804412,8.827778,1643.173333,11.369308,46.100000,8.677309,NaN,8.711236,NaN,7.321601,NaN,NaN,13.642105,NaN,10.106061,8.035714,7.947712,13.375000,8.970190,15.600000,10.039439,NaN,9.903439,NaN,NaN,11.206695,16.275862,10.574713,11.060606,9.325843,15.529101,10.344828,8.333333,NaN,20.325490,9.581746,1579.528333,11.401750
3,Masters Men 2 (50-59),Male,2015.400000,44.100000,8.772490,NaN,8.986142,NaN,7.340580,NaN,NaN,11.168421,NaN,10.090909,8.178571,7.741176,13.527778,9.524390,14.688889,10.220792,NaN,10.055000,NaN,NaN,10.905983,15.517241,10.586207,10.568182,1

In [46]:
# Read in Split Timings
weighted_pb.to_csv('../../data/ws_weighted_pb.csv', encoding = "ISO-8859-1")

In [47]:
# Read in Split Timings
w_pb = pd.read_csv('../../data/ws_weighted_pb.csv', encoding = "ISO-8859-1")

In [48]:
w_pb.iloc[[8]]

,Unnamed: 0,category,gender,Year,runner_age,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,time,AVG_Pace,runner_age_pb,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb,time_pb,AVG_Pace_pb
8,8,Senior Women (23-39),Female,2014.222222,38.0,7.104641,NaN,7.135664,NaN,5.860507,NaN,NaN,9.122807,NaN,7.659933,5.912698,5.647059,9.490741,6.720867,11.45679,7.638064,NaN,6.997619,NaN,NaN,7.179487,10.842912,8.224777,7.857143,7.303371,10.25463,9.310345,6.567568,NaN,14.551743,7.718695,1217.685185,8.244964,38.444444,6.816265,NaN,6.957137,NaN,5.542539,NaN,NaN,7.660819,NaN,7.626263,6.011905,5.30719,9.228395,6.422764,14.450617,7.322149,NaN,6.890212,NaN,NaN,6.99905,9.655172,7.688378,7.359307,NaN,9.62963,NaN,NaN,NaN,22.228214,7.136684,1160.674074,8.506664


####  means

In [49]:
def get_mean_pb(neighbours):
    mean = neighbours.AVG_Pace_pb.mean() 
    return mean
    
# Show sample output    
neighbours = find_nearest_neighbours(10)
get_mean_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


6.982489721344663

In [50]:
def get_weight_mean(neighbours):
    w_mean = neighbours.weighted_pace.sum()
    return w_mean

neighbours = find_nearest_neighbours(10)
get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


6.984209120173829

In [51]:
def euc_get_weight_mean(neighbours):
    ew_mean = neighbours.weighted_pace.sum()
    return ew_mean

neighbours = euc_find_nearest_neighbours(10)
euc_get_weight_mean(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


6.855209508306579

In [52]:
def get_best_npb(neighbours):
    return neighbours.AVG_Pace.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(60)
get_best_npb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


11.557173017002096

##  Best pace row

In [53]:
def best_pace_rec(index):
    query = npb.iloc[index]
    query = pd.DataFrame(query)
    query = query.T
    row_pace = find_nearest_neighbours_pace(index)
    row_pace = row_pace.sort_values(by='AVG_Pace_pb', ascending=True)
    row_pace = row_pace.head(1)
   # row_pace = pd.DataFrame(row_pace)
   # row_pace = row_pace.T
    row_pace = row_pace.filter(like='to')
    row_pace.index = query.index

    return row_pace


best_pace_rec(5)

,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
5,7.349398,NaN,7.303371,NaN,5.869565,NaN,NaN,8.631579,NaN,8.636364,6.785714,6.117647,12.361111,9.512195,17.111111,12.079208,NaN,8.142857,NaN,NaN,9.65812,11.034483,7.931034,8.441558,NaN,11.851852,NaN,NaN,NaN,25.362745,7.357143,7.168675,NaN,6.966292,NaN,5.652174,NaN,NaN,7.684211,NaN,7.575758,5.535714,5.647059,9.583333,6.463415,10.888889,7.623762,NaN,7.142857,NaN,NaN,7.863248,10.689655,7.701149,7.662338,NaN,11.111111,NaN,6.351351,NaN,10.612745,8.769841


In [54]:
pb_top = pd.DataFrame()

In [55]:
pb_top

""


In [56]:
for i in range(pb1.shape[0]):
    pb_top = pb_top.append(best_pace_rec(i), ignore_index=True)

In [57]:
pb_top

,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
0,6.325301,NaN,7.078652,NaN,6.014493,NaN,NaN,9.578947,NaN,10.606061,8.392857,6.235294,14.861111,10.609756,14.000000,8.118812,NaN,7.928571,NaN,NaN,10.256410,15.862069,9.655172,13.506494,NaN,13.148148,NaN,NaN,NaN,29.705882,10.111111,7.931727,NaN,8.239700,NaN,6.159420,NaN,NaN,8.631579,NaN,6.818182,6.428571,6.000000,9.722222,6.829268,12.000000,9.902640,NaN,7.213095,NaN,NaN,7.435897,11.379310,8.160920,NaN,8.314607,NaN,10.517241,NaN,NaN,21.470588,7.857143
1,8.140562,NaN,8.235955,NaN,6.852657,NaN,NaN,9.684211,NaN,10.151515,NaN,12.235294,11.527778,NaN,25.777778,8.288779,NaN,8.020238,NaN,NaN,8.205128,12.413793,9.310345,9.220779,NaN,11.111111,NaN,NaN,NaN,26.176471,7.492063,7.469880,NaN,8.314607,NaN,6.884058,NaN,NaN,9.157895,NaN,8.484848,5.892857,6.117647,10.138889,6.097561,11.333333,7.623762,NaN,6.928571,NaN,NaN,7.692308,14.137931,8.275862,8.571429,NaN,10.185185,NaN,6.486486,NaN,11.936275,10.642857
2,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.912651,NaN,8.882022,NaN,7.101449,NaN,NaN,9.894737,NaN,7.575758,7.321429,6.823529,11.111111,7.439024,12.444444,7.849835,NaN,8.122619,NaN,NaN,7.777778,11.034483,8.735632,NaN,8.539326,NaN,9.655172,NaN,NaN,16.470588,7.317460
3,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.912651,NaN,8.882022,NaN,7.101449,NaN,NaN,9.894737,NaN,7.575758,7.321429,6.823529,11.111111,7.439024,12.444444,7.849835,NaN,8.122619,NaN,NaN,7.777778,11.034483,8.735632,NaN,8.539326,NaN,9.655172,NaN,NaN,16.470588,7.317460
4,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.071429,NaN,NaN,8.717949,12.413793,9.655172,9.870130,NaN,10.925926,NaN,NaN,NaN,24.656863,7.182540,6.807229,NaN,6.966292,NaN,5.579710,NaN,NaN,8.000000,NaN,7.87

In [58]:
pb_top = comb.join(pb_top)

In [59]:
pb_top

,category,gender,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
0,Masters Men 1 (40-49),Male,6.325301,NaN,7.078652,NaN,6.014493,NaN,NaN,9.578947,NaN,10.606061,8.392857,6.235294,14.861111,10.609756,14.000000,8.118812,NaN,7.928571,NaN,NaN,10.256410,15.862069,9.655172,13.506494,NaN,13.148148,NaN,NaN,NaN,29.705882,10.111111,7.931727,NaN,8.239700,NaN,6.159420,NaN,NaN,8.631579,NaN,6.818182,6.428571,6.000000,9.722222,6.829268,12.000000,9.902640,NaN,7.213095,NaN,NaN,7.435897,11.379310,8.160920,NaN,8.314607,NaN,10.517241,NaN,NaN,21.470588,7.857143
1,Masters Men 1 (40-49),Male,8.140562,NaN,8.235955,NaN,6.852657,NaN,NaN,9.684211,NaN,10.151515,NaN,12.235294,11.527778,NaN,25.777778,8.288779,NaN,8.020238,NaN,NaN,8.205128,12.413793,9.310345,9.220779,NaN,11.111111,NaN,NaN,NaN,26.176471,7.492063,7.469880,NaN,8.314607,NaN,6.884058,NaN,NaN,9.157895,NaN,8.484848,5.892857,6.117647,10.138889,6.097561,11.333333,7.623762,NaN,6.928571,NaN,NaN,7.692308,14.137931,8.275862,8.571429,NaN,10.185185,NaN,6.486486,NaN,11.936275,10.642857
2,Masters Men 2 (50-59),Male,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.912651,NaN,8.882022,NaN,7.101449,NaN,NaN,9.894737,NaN,7.575758,7.321429,6.823529,11.111111,7.439024,12.444444,7.849835,NaN,8.122619,NaN,NaN,7.777778,11.034483,8.735632,NaN,8.539326,NaN,9.655172,NaN,NaN,16.470588,7.317460
3,Masters Men 2 (50-59),Male,8.532129,NaN,9.029963,NaN,7.608696,NaN,NaN,12.105263,NaN,11.515152,8.214286,8.000000,13.055556,9.390244,13.777778,9.405941,NaN,10.000000,NaN,NaN,10.341880,14.827586,9.770115,10.259740,NaN,11.851852,NaN,NaN,NaN,26.754902,7.031746,8.912651,NaN,8.882022,NaN,7.101449,NaN,NaN,9.894737,NaN,7.575758,7.321429,6.823529,11.111111,7.439024,12.444444,7.849835,NaN,8.122619,NaN,NaN,7.777778,11.034483,8.735632,NaN,8.539326,NaN,9.655172,NaN,NaN,16.470588,7.317460
4,Senior Women (23-39),Female,6.987952,NaN,7.640449,NaN,5.434783,NaN,NaN,9.473684,NaN,9.090909,6.607143,5.764706,10.138889,6.463415,11.777778,7.821782,NaN,8.07142

In [60]:
# Read in Split Timings
pb_top.to_csv('../../data/ws_pb_top.csv', encoding = "ISO-8859-1")

In [61]:
# Read in Split Timings
pb_top = pd.read_csv('../../data/ws_pb_top.csv', encoding = "ISO-8859-1")

In [62]:
pb_top.iloc[[8]]

,Unnamed: 0,category,gender,Start to Lyon Ridge,Start to Talbot Creek,Lyon Ridge to Red Star Ridge,Talbot Creek to Poppy Trailhead,Red Star Ridge to Duncan Canyon,Poppy Trailhead to Duncan Canyon,Duncan Canyon to Mosquito Ridge,Duncan Canyon to Robinson Flat,Mosquito Ridge to Millers Defeat,Robinson Flat to Millers Defeat,Millers Defeat to Dusty Corners,Dusty Corners to Last Chance,Last Chance to Devils Thumb,Devils Thumb to El Dorado Creek,El Dorado Creek to Michigan Bluff,Michigan Bluff to Foresthill,Foresthill to Dardanelles,Foresthill to Peachstone,Dardanelles to Peachstone,Peachstone to Fords Bar,Peachstone to Rucky Chucky,Rucky Chucky to Green Gate,Green Gate to Auburn Lake Trails,Auburn Lake Trails to Browns Bar,Auburn Lake Trails to Quarry Road,Browns Bar to Highway 49,Quarry Road to Pointed Rocks,Highway 49 to No Hands Bridge,Pointed Rocks to No Hands Bridge,No Hands Bridge to Robie Point,Robie Point to Finish,Start to Lyon Ridge_pb,Start to Talbot Creek_pb,Lyon Ridge to Red Star Ridge_pb,Talbot Creek to Poppy Trailhead_pb,Red Star Ridge to Duncan Canyon_pb,Poppy Trailhead to Duncan Canyon_pb,Duncan Canyon to Mosquito Ridge_pb,Duncan Canyon to Robinson Flat_pb,Mosquito Ridge to Millers Defeat_pb,Robinson Flat to Millers Defeat_pb,Millers Defeat to Dusty Corners_pb,Dusty Corners to Last Chance_pb,Last Chance to Devils Thumb_pb,Devils Thumb to El Dorado Creek_pb,El Dorado Creek to Michigan Bluff_pb,Michigan Bluff to Foresthill_pb,Foresthill to Dardanelles_pb,Foresthill to Peachstone_pb,Dardanelles to Peachstone_pb,Peachstone to Fords Bar_pb,Peachstone to Rucky Chucky_pb,Rucky Chucky to Green Gate_pb,Green Gate to Auburn Lake Trails_pb,Auburn Lake Trails to Browns Bar_pb,Auburn Lake Trails to Quarry Road_pb,Browns Bar to Highway 49_pb,Quarry Road to Pointed Rocks_pb,Highway 49 to No Hands Bridge_pb,Pointed Rocks to No Hands Bridge_pb,No Hands Bridge to Robie Point_pb,Robie Point to Finish_pb
8,8,Senior Women (23-39),Female,6.86747,NaN,6.966292,NaN,5.57971,NaN,NaN,8.0,NaN,7.727273,5.714286,5.529412,9.305556,6.219512,10.666667,7.326733,NaN,7.142857,NaN,NaN,7.264957,10.0,8.275862,7.922078,NaN,11.851852,NaN,4.594595,NaN,9.362745,7.468254,6.807229,NaN,6.966292,NaN,5.57971,NaN,NaN,8.0,NaN,7.878788,6.428571,5.411765,9.027778,6.463415,10.222222,7.227723,NaN,7.214286,NaN,NaN,6.752137,9.310345,7.816092,7.142857,NaN,8.888889,NaN,NaN,NaN,20.588235,7.007937


## Get Best

In [63]:
def get_best_pb(neighbours):
    return neighbours.AVG_Pace_pb.min()
  
# Show sample output    
neighbours = find_nearest_neighbours(5)
get_best_pb(neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


7.9346788643540069

In [64]:
def get_prediction(index, neighbours):
    weight = merge.iloc[index].AVG_Pace/get_best_npb(neighbours)
    best_pb = get_best_pb(neighbours)
    return best_pb * weight

# Show output
neighbours = find_nearest_neighbours(0)
get_prediction(0, neighbours)

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


9.6688286702735891

# Calculate Predictions

In [65]:
def update_rows(row):
    index = int(row['best_pb_prediction'])
    row['query_AVG_Pace'] = merge.iloc[index].AVG_Pace
    row['query_pb'] = merge.iloc[index].AVG_Pace_pb
    neighbours = find_nearest_neighbours(index)
    row['mean_pb_average_pace'] = get_mean_pb(neighbours)
   # row['weighted_mean_pb_ap'] = weighted_pace(neighbours)
  #  row['best_average_pace'] = get_best_npb(neighbours)
    row['best_pb_prediction'] = get_prediction(index, neighbours)
    row['weighted_mean'] = get_weight_mean(neighbours)
    neighbours1 = euc_find_nearest_neighbours(index)
    row['euc_weighted_mean'] = euc_get_weight_mean(neighbours1)
   # row['ranked_mean'] = get_rank_mean(neighbours)
    return row

In [66]:
num_of_runs = merge.time.count()
filler_data = np.arange(0.0, num_of_runs)
df = pd.DataFrame({'best_pb_prediction':filler_data, 'mean_pb_average_pace':filler_data,  'weighted_mean':filler_data,   'euc_weighted_mean':filler_data, 'query_AVG_Pace':filler_data, 'query_pb':filler_data})

df = df.apply(lambda row: update_rows(row), axis=1)
df = df[['query_AVG_Pace', 'query_pb', 'best_pb_prediction', 'mean_pb_average_pace', 'weighted_mean', 'euc_weighted_mean']]
df

C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Cathal Mc Connell\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,11.804720,10.605747,9.668829,10.744975,10.744982,10.741196
1,12.242579,9.051986,9.301479,10.748605,10.737365,10.455589
2,10.913974,13.559780,9.043635,11.401750,11.402140,11.241669
3,11.844286,11.303364,9.881917,11.274727,11.274394,11.425228
4,9.080386,9.710111,9.063317,8.450547,8.448198,8.442814
5,10.454235,9.710111,8.692657,8.779989,8.774615,8.772191
6,9.583453,8.170402,8.833377,8.677684,8.673364,8.672889
7,8.623698,8.170402,8.607488,8.597329,8.597770,8.587025
8,7.789676,8.170402,8.509161,8.506664,8.510561,8.514494
9,7.624734,7.846284,6.602893,7.409404,7.409647,7.115347


In [67]:
df.iloc[[66]]

,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
66,10.721718,9.577031,8.617012,9.102699,9.085322,9.081403


Comb dataframe contains the information of the gender and category of each runner. This will be used in a later notebook to test how accurate the predictions are in comparison to male, female, senior men, etc...

In [68]:
comb.join(df)

,category,gender,query_AVG_Pace,query_pb,best_pb_prediction,mean_pb_average_pace,weighted_mean,euc_weighted_mean
0,Masters Men 1 (40-49),Male,11.804720,10.605747,9.668829,10.744975,10.744982,10.741196
1,Masters Men 1 (40-49),Male,12.242579,9.051986,9.301479,10.748605,10.737365,10.455589
2,Masters Men 2 (50-59),Male,10.913974,13.559780,9.043635,11.401750,11.402140,11.241669
3,Masters Men 2 (50-59),Male,11.844286,11.303364,9.881917,11.274727,11.274394,11.425228
4,Senior Women (23-39),Female,9.080386,9.710111,9.063317,8.450547,8.448198,8.442814
5,Senior Women (23-39),Female,10.454235,9.710111,8.692657,8.779989,8.774615,8.772191
6,Senior Women (23-39),Female,9.583453,8.170402,8.833377,8.677684,8.673364,8.672889
7,Senior Women (23-39),Female,8.623698,8.170402,8.607488,8.597329,8.597770,8.587025
8,Senior Women (23-39),Female,7.789676,8.170402,8.509161,8.506664,8.510561,8.514494
9,Senior Men (23-39),Male,7.624734,7.846284,6.602893,7.409404,7.409647,7.115347


In [69]:
df = comb.join(df)

In [70]:
df.to_csv('../../data/ws_pace_predictions.csv')